In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()

script_dir = os.path.dirname(os.path.abspath(""))
openagent_dir = os.path.dirname(script_dir)
sys.path.append(openagent_dir)

In [2]:
from openagent import compiler
from openagent.llms._openai import OpenAI
from openagent.helpers.youtubeSearch import YouTubeSearchTool
from openagent.helpers.youtubeTranscript import YoutubeTranscriptReader

import json

tools = [YouTubeSearchTool(), YoutubeTranscriptReader()]

c:\Users\DELL\anaconda3\envs\openagent\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [3]:
llm = OpenAI(model="gpt-3.5-turbo-16k")

In [11]:
temp = '''
{{#system~}}
You are a web assistant who has access to youtube based tool
you have access to the following tools:-
{{tool}}
{{~#each tool}}
{{this}}
{{/each}}
{{~/system}}

{{#user~}}
answer this question - {{query}} by following this steps
first think whether you need to use the tool or not. answer this in one word either <<Yes>> or <<No>>.
{{~/user}}

{{#assistant~}}
{{gen 'tool_use'}}
{{~/assistant}}

{{#user~}}
If the answer is Yes then call the tool using the following format '{"index":[index of the tool to be used in the tools list], "query":[query to be passed]'
If the answer is No, answer to the {{query}} itself.
{{~/user}}

{{#assistant~}}
{{gen 'action' temperature=0 max_tokens=500}}
{{#if (tool_use)=="Yes"}}
Tool Response: {{(tool_func action)}}
{{/if}}

Formatting the response into full answer
{{~/assistant}}

{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}

'''

In [15]:
def tool_use(var:dict, tools=tools):
    var = json.loads(var)
    # print(tools[int(var['index'])])
    return tools[int(var['index'])].run(var['query'])

prompt = compiler(template=temp, llm=llm,  tool=tools, tool_func = tool_use, stream=False, silent=True)

response = prompt(query='give video links for Amitabh Bachan')

response['response']

'Here are some video links for Amitabh Bachchan:\n\n1. [Amitabh Bachchan - The Ultimate Superstar](https://www.youtube.com/watch?v=yRkqXDiOl1I&pp=ygUQQW1pdGFiaCBCYWNoY2hhbg%3D%3D)\n2. [Amitabh Bachchan - The Living Legend](https://www.youtube.com/watch?v=vB2m1uRF30c&pp=ygUQQW1pdGFiaCBCYWNoY2hhbg%3D%3D)'

In [7]:
tool = YouTubeSearchTool()

tool.run("Lil Nas X")

"['/watch?v=UTHLKHL_whs&pp=ygUJTGlsIE5hcyBY', '/watch?v=r7qovpFAGrQ&pp=ygUJTGlsIE5hcyBY']"